In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/03/01 01:35:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [13]:
spark.version

'3.3.2'

In [9]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

--2025-03-01 01:45:41--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 54.230.209.126, 54.230.209.140, 54.230.209.200, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|54.230.209.126|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64346071 (61M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2024-10.parquet’

yellow_tripdata_202 100%[===================>]  61.36M   165MB/s    in 0.4s    

2025-03-01 01:45:42 (165 MB/s) - ‘yellow_tripdata_2024-10.parquet’ saved [64346071/64346071]



In [10]:
df = spark.read \
    .option("header", "true") \
    .parquet('yellow_tripdata_2024-10.parquet')

In [11]:
df = df.repartition(4)

df.write.parquet('yellow/2024/10')

In [12]:
!ls -lh yellow/2024/10

total 97M
-rw-r--r-- 1 aa aa   0 Mar  1 01:47 _SUCCESS
-rw-r--r-- 1 aa aa 25M Mar  1 01:47 part-00000-adff9000-1d60-420a-b42d-32b0d1b4e5d4-c000.snappy.parquet
-rw-r--r-- 1 aa aa 25M Mar  1 01:47 part-00001-adff9000-1d60-420a-b42d-32b0d1b4e5d4-c000.snappy.parquet
-rw-r--r-- 1 aa aa 25M Mar  1 01:47 part-00002-adff9000-1d60-420a-b42d-32b0d1b4e5d4-c000.snappy.parquet
-rw-r--r-- 1 aa aa 25M Mar  1 01:47 part-00003-adff9000-1d60-420a-b42d-32b0d1b4e5d4-c000.snappy.parquet


In [15]:
df.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'Airport_fee']

In [23]:
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [21]:
df.createOrReplaceTempView('trips')

In [17]:
from pyspark.sql import functions as F

In [25]:
spark.sql("""
SELECT COUNT(*)
FROM trips
WHERE CAST(tpep_pickup_datetime AS DATE) = '2024-10-15'
""").show()

+--------+
|count(1)|
+--------+
|  128893|
+--------+



In [24]:
spark.sql("""
SELECT (CAST(tpep_dropoff_datetime AS BIGINT) - CAST(tpep_pickup_datetime AS BIGINT)) / 3600 AS trip_duration_hours
FROM trips
ORDER BY trip_duration_hours DESC
LIMIT 1
""").show()

+-------------------+
|trip_duration_hours|
+-------------------+
| 162.61777777777777|
+-------------------+



In [26]:
!wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

--2025-03-01 02:10:24--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 54.230.209.72, 54.230.209.126, 54.230.209.200, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|54.230.209.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12331 (12K) [text/csv]
Saving to: ‘taxi_zone_lookup.csv.1’

taxi_zone_lookup.cs 100%[===================>]  12.04K  --.-KB/s    in 0.003s  

2025-03-01 02:10:25 (4.68 MB/s) - ‘taxi_zone_lookup.csv.1’ saved [12331/12331]



In [27]:
taxi_zone = spark.read \
    .option("header", "true") \
    .csv('taxi_zone_lookup.csv')

taxi_zone.createOrReplaceTempView('zone')

In [28]:
taxi_zone.printSchema()

root
 |-- LocationID: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [31]:
spark.sql("""
SELECT COUNT(*) AS trip_count, z.Zone
FROM trips t
LEFT JOIN zone z
ON CAST(t.PULocationID AS STRING) = z.LocationID
GROUP BY z.Zone
ORDER BY trip_count
LIMIT 1;
""").show()

+----------+--------------------+
|trip_count|                Zone|
+----------+--------------------+
|         1|Governor's Island...|
+----------+--------------------+

